In [26]:
tol=1E-9
maxiter = 1000000
sigma = 0.01

In [27]:
seed = 777
nbX=10
nbY=8
set.seed(seed)
Phi=matrix(runif(nbX*nbY),nrow=nbX)
p=rep(1/nbX,nbX)
q=rep(1/nbY,nbY)

In [28]:
library(Matrix)

In [29]:
library(gurobi)

In [30]:
A1 = kronecker(matrix(1,1,nbY),sparseMatrix(1:nbX,1:nbX))
A2 = kronecker(sparseMatrix(1:nbY,1:nbY),matrix(1,1,nbX))
A = rbind2(A1,A2)

In [31]:
result   = gurobi ( list(A=A,obj=c(Phi),modelsense="max",rhs=c(p,q),sense="="), params=list(OutputFlag=0) ) 

In [7]:
str(result)

List of 13
 $ status      : chr "OPTIMAL"
 $ runtime     : num 0
 $ itercount   : num 22
 $ baritercount: int 0
 $ nodecount   : num 0
 $ objval      : num 0.869
 $ x           : num [1:80] 0 0 0 0.1 0 0 0 0 0.025 0 ...
 $ slack       : num [1:18] 0 0 0 0 0 0 0 0 0 0 ...
 $ pi          : num [1:18] -0.2919 -0.3644 -0.2417 -0.0713 -0.1584 ...
 $ rc          : num [1:80] -0.0865 -0.2097 -0.4795 0 -0.2126 ...
 $ vbasis      : int [1:80] -1 -1 -1 0 -1 -1 -1 -1 0 -1 ...
 $ cbasis      : int [1:18] -1 -1 -1 -1 -1 -1 -1 0 -1 -1 ...
 $ objbound    : num 0.869


IFPP

In [32]:
cont = TRUE
  iter = 0
  
  K=exp(Phi/sigma)
  B=rep(1,nbY)

In [33]:
  while (cont) {
    iter = iter+1
    A = p / c(K %*% B)
    pi = t(t(K*A)*B)
    if ((max(abs(apply(X=pi,FUN=sum,MARGIN=2) /q-1)) < tol) | (iter>maxiter)) {cont=FALSE}
    B = q / c(A %*% K)
  }

In [35]:
sigma = 0.001

In [37]:
 ptm=proc.time()
iter = 0
  cont=TRUE
  v = rep(0,nbY)
  mu = - sigma * log(p)
  nu = - sigma * log(q)
  uprec = -Inf
  while(cont)
  {
    #print(iter)
    iter = iter+1
    vstar = apply(t(t(Phi) - v), 1, max)
    
    u=mu + vstar + sigma * log( apply(exp( (Phi - matrix(v,nbX,nbY,byrow=T) - vstar)/sigma) ,1, sum) )
    error = max(abs(u-uprec))
    uprec = u
    
    ustar = apply( Phi - u , 2, max)
    v = nu + ustar + sigma * log(apply(exp( (Phi - u - matrix(ustar,nbX,nbY,byrow=T) )/sigma) ,2, sum))
    
    if( (error<tol) | (iter >= maxiter)) {cont=FALSE}
    
  }
  pi = exp( ( Phi - u - matrix(v,nbX,nbY,byrow=T) ) / sigma )
  val = sum(pi*Phi) - sigma* sum(pi*log(pi))
  time = proc.time()-ptm
  
  if (iter >= maxiter ) 
  {print('Maximum number of iterations reached in IPFP2.')
  } else {
    print(paste0("IPFP2 converged in ",iter, " steps and ", time[1], "s."))
    print(paste0("Value of the problem (IPFP2) = ",val))
    print(paste0("Sum(pi*Phi) (IPFP2) = ",sum(pi*Phi)))
    print(u[1:nrow]-u[nrow])
    print(v[1:ncol]+u[nrow])
  }

[1] "IPFP2 converged in 485768 steps and 198.41s."
[1] "Value of the problem (IPFP2) = NaN"
[1] "Sum(pi*Phi) (IPFP2) = 0.869151724470475"


ERROR: Error in 1:nrow: NA/NaN Argument
